# MODELO ACEPTACIÓN/RECHAZO

In [46]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

## --- Conexión a la base de datos ---

In [47]:
db_file = 'data/vacaciones_empresa.db'
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

## --- Carga de datos desde las tablas ---

In [48]:
empleados_df = pd.read_sql_query("SELECT * FROM empleados", conn)
solicitudes_df = pd.read_sql_query("SELECT * FROM solicitudes_vacaciones", conn)
calendario_df = pd.read_sql_query("SELECT * FROM calendario_laboral", conn)
politicas_df = pd.read_sql_query("SELECT * FROM politicas_vacaciones", conn, index_col='departamento')

## --- Fusión de DataFrames ---

In [49]:
# Fusionar solicitudes con información del empleado
data_df = pd.merge(solicitudes_df, empleados_df, on='id_empleado', suffixes=('_solicitud', '_empleado'))

In [50]:
# Fusionar con información del calendario (opcional, basado en la fecha de inicio de la solicitud)
data_df = pd.merge(data_df, calendario_df, left_on='fecha_inicio', right_on='fecha', how='left')
data_df.drop(columns=['fecha'], inplace=True)

## --- Preprocesamiento de datos ---

In [51]:
# 1. Manejo de valores faltantes
data_df['motivo_rechazo'].fillna('', inplace=True) # Los motivos de rechazo faltantes significan que fue aceptada

/tmp/ipykernel_42409/133363434.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_df['motivo_rechazo'].fillna('', inplace=True) # Los motivos de rechazo faltantes significan que fue aceptada


In [52]:
# 2. Creación de características
data_df['duracion_dias'] = data_df['duracion_dias'].astype(int)
data_df['antiguedad_meses'] = data_df['antiguedad_meses'].astype(int)
data_df['mes_solicitud'] = data_df['mes_solicitud'].astype(int)

# Convertir columnas de fecha a datetime
data_df['fecha_contratacion'] = pd.to_datetime(data_df['fecha_contratacion'])
data_df['fecha_inicio'] = pd.to_datetime(data_df['fecha_inicio'])
data_df['fecha_fin'] = pd.to_datetime(data_df['fecha_fin'])
data_df['fecha_solicitud'] = pd.to_datetime(data_df['fecha_solicitud'])

# Calcular la diferencia en días entre la fecha de solicitud y la fecha de inicio
data_df['dias_anticipacion'] = (data_df['fecha_inicio'] - data_df['fecha_solicitud']).dt.days

# Extraer información temporal adicional
data_df['dia_semana_inicio'] = data_df['fecha_inicio'].dt.day_name()
data_df['mes_inicio'] = data_df['fecha_inicio'].dt.month

In [53]:
# 3. Codificación de variables categóricas
categorical_cols = ['departamento', 'puesto', 'dia_semana_solicitud', 'genero', 'nivel_educacion', 'dia_semana', 'temporada', 'dia_semana_inicio']
label_encoders = {}
for col in categorical_cols:
    if col in data_df.columns: # Asegurarse de que la columna existe
        label_encoders[col] = LabelEncoder()
        data_df[col] = label_encoders[col].fit_transform(data_df[col].astype(str))
    else:
        print(f"Advertencia: La columna categórica '{col}' no se encontró en el DataFrame.")

# Codificar la variable objetivo 'estado' (Aceptada: 1, Rechazada: 0)
data_df['estado_encoded'] = np.where(data_df['estado'] == 'Aceptada', 1, 0)

In [54]:
# 4. Normalización de variables numéricas
numerical_cols = ['antiguedad_meses', 'edad', 'duracion_dias', 'periodo_anio', 'mes_solicitud', 'dias_anticipacion', 'mes_inicio']
scaler = StandardScaler()
# Asegurarse de que solo se normalizan las columnas numéricas existentes
numerical_cols_present = [col for col in numerical_cols if col in data_df.columns]
if numerical_cols_present:
    data_df[numerical_cols_present] = scaler.fit_transform(data_df[numerical_cols_present])
else:
    print("Advertencia: No se encontraron columnas numéricas para normalizar.")

## --- Preparación de los datos para el modelo de clasificación (Aceptación/Rechazo) ---

In [55]:
X = data_df.drop(columns=['id_solicitud', 'id_empleado', 'nombre', 'fecha_contratacion', 'fecha_inicio', 'fecha_fin', 'fecha_solicitud', 'estado', 'motivo_rechazo', 'estado_encoded', 'correo_electronico'])
y = data_df['estado_encoded']

In [56]:
print("Columnas en X antes de la división:", X.columns)

Columnas en X antes de la división: Index(['duracion_dias', 'periodo_anio', 'mes_solicitud',
       'dia_semana_solicitud', 'departamento', 'puesto', 'antiguedad_meses',
       'edad', 'genero', 'nivel_educacion', 'es_festivo', 'dia_semana',
       'temporada', 'dias_anticipacion', 'dia_semana_inicio', 'mes_inicio'],
      dtype='object')


## --- División de datos en entrenamiento y prueba ---

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## --- Entrenamiento del modelo de clasificación (Random Forest) ---

In [58]:
model_acceptance = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
model_acceptance.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=42)

## --- Evaluación del modelo de clasificación ---

In [59]:
y_pred_acceptance = model_acceptance.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_acceptance)
report = classification_report(y_test, y_pred_acceptance)

print("--- Resultados del Modelo de Predicción de Aceptación/Rechazo ---")
print(f"Precisión del modelo: {accuracy:.4f}")
print("Reporte de Clasificación:\n", report)

--- Resultados del Modelo de Predicción de Aceptación/Rechazo ---
Precisión del modelo: 0.7465
Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.79      0.79      0.79      1221
           1       0.67      0.68      0.68       779

    accuracy                           0.75      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.75      0.75      0.75      2000



## --- Cierre de la conexión a la base de datos ---

In [60]:
conn.close()